In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset , mnist_info = tfds.load(name = 'mnist', with_info = True , as_supervised = True)

In [6]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)



buffer_size = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)



batch_size = 200

train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))

In [7]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape = (28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'),
                            tf.keras.layers.Dense(output_size, activation ='softmax')
                            ])

In [10]:
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = custom_optimizer, loss = 'sparse_categorical_crossentropy', metrics =['accuracy'])

In [12]:
num_epochs = 6
validation_steps = num_validation_samples

model.fit(train_data, epochs = num_epochs, 
          validation_data = (validation_inputs, validation_targets), validation_steps = validation_steps , 
          verbose = 2) 

Epoch 1/6
270/270 - 9s - loss: 0.0105 - accuracy: 0.9962 - val_loss: 0.0091 - val_accuracy: 0.9973
Epoch 2/6
270/270 - 9s - loss: 0.0076 - accuracy: 0.9975 - val_loss: 0.0100 - val_accuracy: 0.9970
Epoch 3/6
270/270 - 10s - loss: 0.0071 - accuracy: 0.9976 - val_loss: 0.0100 - val_accuracy: 0.9963
Epoch 4/6
270/270 - 11s - loss: 0.0100 - accuracy: 0.9965 - val_loss: 0.0108 - val_accuracy: 0.9970
Epoch 5/6
270/270 - 11s - loss: 0.0087 - accuracy: 0.9970 - val_loss: 0.0196 - val_accuracy: 0.9943
Epoch 6/6
270/270 - 11s - loss: 0.0080 - accuracy: 0.9973 - val_loss: 0.0080 - val_accuracy: 0.9973


In [13]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 1s 907ms/step - loss: 0.0898 - accuracy: 0.980 - 1s 915ms/step - loss: 0.0898 - accuracy: 0.9803

In [14]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

Test loss: 0.09. Test accuracy: 98.03%
